In [14]:
import os
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

In [2]:
src = 'data/train'

texts = []
labels = []

for label in ['neg', 'pos']:
    
    dir_name = os.path.join(src, label)
    
    for fname in os.listdir(dir_name):
        
        if fname[-4:] == '.txt':
            
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            
            if label == 'neg':
                
                labels.append(0)
            else:
                
                labels.append(1)

In [3]:
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

In [4]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [5]:
word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'he': 26,
 'be': 27,
 'one': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'who': 34,
 'so': 35,
 'from': 36,
 'like': 37,
 'her': 38,
 'or': 39,
 'just': 40,
 'about': 41,
 "it's": 42,
 'out': 43,
 'has': 44,
 'if': 45,
 'some': 46,
 'there': 47,
 'what': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'she': 56,
 'even': 57,
 'my': 58,
 'would': 59,
 'which': 60,
 'only': 61,
 'story': 62,
 'really': 63,
 'see': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'were': 68,
 'me': 69,
 'well': 70,
 'than': 71,
 'we': 72,
 'much': 73,
 'been': 74,
 'bad': 75,
 'get': 76,
 'will': 77,
 'do': 78,
 'also': 79,
 'into': 80,
 'people': 81,
 'other': 82,
 '

In [6]:
data = pad_sequences(sequences, maxlen = maxlen)
labels = np.asarray(labels)
print(data.shape)
print(labels.shape)

(25000, 100)
(25000,)


In [7]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [8]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [9]:
glove_dir = 'glove.6B'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
len(embeddings_index)

400000

In [13]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [15]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                320032    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [18]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data = (x_val, y_val))

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 2s 8ms/sample - loss: 1.6063 - accuracy: 0.4700 - val_loss: 0.6930 - val_accuracy: 0.5123
Epoch 2/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.6680 - accuracy: 0.6450 - val_loss: 0.6920 - val_accuracy: 0.5223
Epoch 3/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.6289 - accuracy: 0.6750 - val_loss: 0.9710 - val_accuracy: 0.5002
Epoch 4/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.5886 - accuracy: 0.6800 - val_loss: 1.0103 - val_accuracy: 0.5003
Epoch 5/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.3744 - accuracy: 0.8950 - val_loss: 1.1642 - val_accuracy: 0.5008
Epoch 6/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.3872 - accuracy: 0.8400 - val_loss: 0.7065 - val_accuracy: 0.5615
Epoch 7/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.2012 - acc